<a href="https://colab.research.google.com/github/felixzhao/title_catgories_classification/blob/main/Job_title_classification_V3_3_level_2_categories_hierarchy_embedding_cossim_model_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install dependency

In [ ]:
! pip install sentence-transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 KB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 44.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=7125e6ae0a2b4666a01dc86b9a8044b0b3980f0adc19a798a8294c43430fd39a
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


# load packages

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from collections import defaultdict
import json

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report

from sentence_transformers import SentenceTransformer, util
from sklearn import metrics

load data
This work loads data from google drive. Please put the files into "trademe_data" folder under the root of google drive.

Then run the following code to mount the google drive in colab.

You can follow the guide. https://towardsdatascience.com/different-ways-to-connect-google-drive-to-a-google-colab-notebook-pt-1-de03433d2f7a

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
root_path = "drive/MyDrive/trademe_data/"

Mounted at /content/drive


# load  data

In [ ]:
raw_path = f"{root_path}raw.csv"
unique_categories_path = f"{root_path}level2_unique_categories_mapping.txt"
level_2_mapping_level_1_categories_path = f"{root_path}level2_to_level1_categories_mapping.txt"
level_2_mapping_level_3_categories_path = f"{root_path}level2_to_level3_categories_mapping.txt"

In [ ]:
raw_df = pd.read_csv(raw_path)
raw_df.head()

,Unnamed: 0,x_title,y_cat_id,cat_1,cat_2,cat_3,x_title_feature
0,0,unqualified asbestos remover,5192,trades & services,labourers,labourers,unqualified asbestos remover
1,1,senior test analyst,5123,it,testing,testing,senior test analyst
2,2,ict trainer / supervisor,6894,education,tutoring & training,tutoring & training,ict trainer supervisor
3,3,automotive specialists *multi faceted position*,5197,trades & services,technicians,technicians,automotive specialists multi faceted position
4,4,business analyst,5114,it,business & systems analysts,business & systems analysts,business analyst


In [ ]:
c2_uni_m_df = json.load(open(unique_categories_path))
print(len(c2_uni_m_df.keys()))
list(c2_uni_m_df.keys())[:3]

149


['account management', 'accounts administrators', 'accounts payable']

In [ ]:
c2_uni_m_df['account management']

5164

In [ ]:
c1_dict = json.load(open(level_2_mapping_level_1_categories_path))
print(len(c1_dict.keys()))
list(c1_dict.keys())[:3]

10


['administration', 'analysts', 'architects']

In [ ]:
c3_dict = json.load(open(level_2_mapping_level_3_categories_path))
print(len(c3_dict.keys()))
list(c3_dict.keys())[:3]

3


['accountants', 'drivers & couriers', 'nursing & midwifery']

# Train & test split

In [ ]:
X_train, X_test , y_train, y_test = train_test_split(raw_df['x_title_feature'],   raw_df[['y_cat_id','cat_1','cat_2','cat_3']],  test_size=0.2,   random_state=123    )

In [ ]:
type(X_train)

pandas.core.series.Series

In [ ]:
train_df = pd.concat([X_train, y_train], axis=1)
train_df.head(2)

,x_title_feature,y_cat_id,cat_1,cat_2,cat_3
9642,kiwifruit orchards workers want,5016,"agriculture, fishing & forestry",farming,farming
17924,orchard hand,5019,"agriculture, fishing & forestry",horticulture,horticulture


In [ ]:
y_test_list = y_test['y_cat_id'].to_list()
y_test_list[:2]

[5141, 6909]

# model class

In [ ]:

class PredictModel:
    def __init__(self,corpus_dict, model):
        
        self.trait_embedding = {}
        self.model = model

        for trait, sentences in list(corpus_dict.items()):
            print(trait)
            self.trait_embedding[trait] = self.model.encode(
                sentences, convert_to_tensor=True
            )

    
            

    def predict(self, input):
        input_embeddings = self.model.encode(input, convert_to_tensor=True)
        trait_score = {}
        max_trait = ""
        max_score = -float("inf")

        for key in self.trait_embedding.keys():
            similar_scores = (
                util.cos_sim(self.trait_embedding[key], input_embeddings).numpy().max()
            )

            # print(key, similar_scores)
            trait_score[key] = similar_scores
            # max_score = max(similar_scores, max_score)
            if similar_scores > max_score:
                max_trait = key
                max_score = similar_scores
        # print(max_trait, max_score)
        # return max_trait, trait_score
        return max_trait

    # def infer(self, inputs, show_log=False):
    #     predict_labels = []
    #     print(type(inputs))
    #     print(len(inputs))
    #     for x in inputs:
    #         max_trait, trait_score = self.predict(x)
    #         if show_log:
    #             print(max_trait, trait_score)
    #         predict_labels.append(max_trait)
    #     return predict_labels


In [ ]:
class LeavePredictModelHandler:
    def __init__(self, df, hierarchy_dict, model, category_level="cat_3"):
        self.hierarchy_dict = hierarchy_dict
        self.category_level = category_level
        corpus_dict_list = self._get_leaves_dicts(df)
        self.predict_models = {}
        for k, v in corpus_dict_list.items():
            self.predict_models[k] = PredictModel(v, model)

    def _get_category_dict(self, df, col_data='x_title_feature'):
        groups = df[[self.category_level, col_data]].groupby(self.category_level)
        res = {key1: group[col_data].to_list() for key1, group in groups}
        return res
    
    def _get_leaves_dicts(self, df, col_data='x_title_feature'):
        leaves_dicts = defaultdict(list)

        for k, v in self.hierarchy_dict.items():
            cat_list = c3_keys = [x[1] for x in v]
            cat_df = df[[self.category_level, col_data]][(df[self.category_level].isin(cat_list))]
            leaves_dicts[k] = self._get_category_dict(cat_df)
        return leaves_dicts

    def get_leaves_node_list(self):
        return list(self.hierarchy_dict.keys())

    def get_leaves_name_id_mapping(self, node_name):
        return {v:k for (k,v) in self.hierarchy_dict[node_name]}

    def get_predict_model(self, node_name):
        return self.predict_models[node_name]

    def get_predict_models(self):
        return self.predict_models

In [ ]:
class PredictPipeline:
    def __init__(self, train_df, level_1_hierarchy_dict, level_3_hierarchy_dict, unique_categories_mapping):
        self.df = train_df
        self.unique_categories_mapping = unique_categories_mapping
        self.level_1_hierarchy_dict = level_1_hierarchy_dict
        self.level_3_hierarchy_dict = level_3_hierarchy_dict
        self.model = SentenceTransformer("all-MiniLM-L6-v2")
        self.level_2_model = None
        self.level_3_models = None
        self.level_1_models = None

    def _get_category_dict(self, df, category_level, col_data='x_title_feature'):
        groups = df[[category_level, col_data]].groupby(category_level)
        res = {key1: group[col_data].to_list() for key1, group in groups}
        return res

    def train(self):
        print('train level 2:')
        corpus_dict = self._get_category_dict(self.df, 'cat_2')
        self.level_2_model = PredictModel( corpus_dict = corpus_dict, model = self.model)
        print('train level 3:')
        self.level_3_models = LeavePredictModelHandler(df = self.df, hierarchy_dict = self.level_3_hierarchy_dict, category_level="cat_3", model=self.model)
        print('train level 1:')
        self.level_1_models = LeavePredictModelHandler(self.df, self.level_1_hierarchy_dict, category_level="cat_1", model=self.model)
    
    def _level_predict(self, query, level_model, c2_pred):
        node_model = level_model.predict_models[c2_pred]
        pred = node_model.predict(query)
        return level_model.get_leaves_name_id_mapping(c2_pred)[pred]

    def predict(self, query):
        c2_pred = self.level_2_model.predict(query)
        hit_level = 'c2'
        if c2_pred in self.level_3_models.predict_models:
            return self._level_predict(query = query, level_model = self.level_3_models, c2_pred = c2_pred)
            # c3_model = self.level_3_models.get_predict_model(c2_pred)
            # pred = c3_model.predict(query)
            # return c3_model.get_leaves_name_id_mapping(c2_pred)[pred]
        elif c2_pred in self.level_1_models.predict_models:
            return self._level_predict(query = query, level_model = self.level_1_models, c2_pred = c2_pred)
            # c1_model = self.level_1_models.get_predict_model(c2_pred)
            # pred = c1_model.predict(query)
            # return c1_model.get_leaves_name_id_mapping(c2_pred)[pred]
        else:
            return -1

        return self.unique_categories_mapping[c2_pred]


    def batch_predict(self, queries):
        predicts = []
        for q in queries:
            predicts.append(self.predict(q))
        return predicts


# Inference

In [ ]:
pipeline = PredictPipeline(train_df = train_df, level_1_hierarchy_dict = c1_dict, level_3_hierarchy_dict = c3_dict, unique_categories_mapping= c2_uni_m_df)

In [ ]:
pipeline.train()

train level 2:
account management
accountants
accounts administrators
accounts payable
accounts receivable
administration
advertising
air con. & refrigeration
analysts
architects
area managers
assistant store managers
au pairs & nannies
automotive technician
bar staff & baristas
beautician
boat building
boilermakers & welders
brand & product management
building & carpentry
building services
business & systems analysts
business development manager
butchers & bakers
buying
call centre
caregiving
central government
chefs
civil & structural
cleaning
client services
commercial sales & leasing
communications & pr
community & social services
consultancy & valuation
corporate & institutional banking
credit & lending
customer-facing
data entry
data warehousing & bi
database
defence
dentistry
department managers
design
diesel mechanic
digital marketing
direct marketing
doctors & specialists
drafting
drivers & couriers
ea, pa & secretarial
early childhood
electrical
energy
environmental
estimatio

In [ ]:
y_pred = pipeline.batch_predict(X_test)

In [ ]:
# pipeline.level_3_models.predict_models #.get_predict_models()

In [ ]:
# pipeline.level_1_models.get_leaves_node_list()

In [ ]:
# pipeline.level_1_models.get_predict_model('electrical')

In [ ]:
y_test

,y_cat_id,cat_1,cat_2,cat_3
12136,5141,manufacturing & operations,storepersons & warehousing,storepersons & warehousing
16812,6909,transport & logistics,drivers & couriers,truck ht
10072,6909,transport & logistics,drivers & couriers,truck ht
5850,5029,"banking, finance & insurance",insurance,insurance
4320,5085,healthcare,community & social services,community & social services
...,...,...,...,...
7481,5175,retail,retail assistants,retail assistants
154,5173,retail,merchandising,merchandising
8627,5101,hospitality & tourism,reception & front desk,reception & front desk
4565,6891,construction & roading,machine operators,machine operators


# Eval Model

In [ ]:
print('eval Accuracy:',metrics.accuracy_score(y_test_list, y_pred))

eval Accuracy: 0.0995


In [ ]:
print(classification_report(y_test_list, y_pred))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         0
         621       0.60      0.46      0.52        13
         674       0.00      0.00      0.00         3
         728       0.00      0.00      0.00        18
         733       0.00      0.00      0.00        19
         748       0.00      0.00      0.00         5
         749       0.00      0.00      0.00         6
         751       0.00      0.00      0.00        10
         752       0.00      0.00      0.00         6
         755       0.00      0.00      0.00         3
         759       0.00      0.00      0.00         3
         761       0.00      0.00      0.00        30
         763       0.00      0.00      0.00         4
         764       0.00      0.00      0.00        11
         765       0.00      0.00      0.00         2
         766       0.00      0.00      0.00         3
         768       0.00      0.00      0.00        13
         883       0.00    

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1

# Practice

In [ ]:
test_c3_dict = {'accountants': [(621, 'business advisory services'),
  (6882, 'assistant accountants'),
  (6883, 'audit'),
  (6884, 'financial accounting & reporting'),
  (6885, 'general / ca'),
  (6886, 'management accountants'),
  (6887, 'tax')]}

In [ ]:
test_c1_dict = {'administration': [(5083, 'healthcare'), (5156, 'office & administration')]}

In [ ]:
pipeline = PredictPipeline(train_df = train_df, level_1_hierarchy_dict = test_c1_dict, level_3_hierarchy_dict = test_c3_dict, unique_categories_mapping= c2_uni_m_df)

In [ ]:
pipeline.train()

train level 2:
account management
accountants
accounts administrators
accounts payable
accounts receivable
administration
advertising
air con. & refrigeration
analysts
architects
area managers
assistant store managers
au pairs & nannies
automotive technician
bar staff & baristas
beautician
boat building
boilermakers & welders
brand & product management
building & carpentry
building services
business & systems analysts
business development manager
butchers & bakers
buying
call centre
caregiving
central government
chefs
civil & structural
cleaning
client services
commercial sales & leasing
communications & pr
community & social services
consultancy & valuation
corporate & institutional banking
credit & lending
customer-facing
data entry
data warehousing & bi
database
defence
dentistry
department managers
design
diesel mechanic
digital marketing
direct marketing
doctors & specialists
drafting
drivers & couriers
ea, pa & secretarial
early childhood
electrical
energy
environmental
estimatio

In [ ]:
pipeline.predict('accountant - retail')

621

In [ ]:
pipeline.predict('account manager')

5164